In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import multiprocessing as mp
tqdm.pandas()

column = 'claim_fulltext' # 'title', 'abstract' or 'claim_fulltext'

In [3]:
# df_keywords_titles = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/cpc_yake_keywords_list_noun_chunks_embeddings.json')
# df_keywords_titles = pd.read_json(f"/home/thiesen/Documents/Cleantech_Concepts/cleantech_keywords_similarity_015_co_occurrence_025_title.json")
df_keywords_titles = pd.read_json(f"/home/thiesen/Documents/Cleantech_Concepts/cleantech_keywords_similarity_005_co_occurrence_025_abstract.json")

# Convert columns 'keyword_patentsberta_embedding', 'keyword_climatebert_embedding', 'keyword_bertforpatents_embedding' to numpy arrays
# df_keywords_titles['keyword_yake_patentsberta_embedding'] = df_keywords_titles['keyword_yake_patentsberta_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
# df_keywords_titles['keyword_yake_climatebert_embedding'] = df_keywords_titles['keyword_yake_climatebert_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_titles['keyword_yake_lemma_bertforpatents_embedding'] = df_keywords_titles['keyword_yake_lemma_bertforpatents_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))

100%|██████████| 4777/4777 [00:00<00:00, 44088.30it/s]


In [4]:
df_keywords_list = pd.read_json(f'/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/df_keywords_list_agg_uspto_epo_rel_embeddings_{column}_noun_chunks.json')

# Convert columns 'keyword_yake_patentsberta_embedding', 'keyword_yake_climatebert_embedding', 'keyword_yake_bertforpatents_embedding' to numpy arrays
# df_keywords_list[f'keywords_yake_{column}_patentsberta_embedding'] = df_keywords_list[f'keywords_yake_{column}_patentsberta_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
df_keywords_list[f'keywords_yake_{column}_bertforpatents_embedding'] = df_keywords_list[f'keywords_yake_{column}_bertforpatents_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))
# df_keywords_list[f'keywords_yake_{column}_climatebert_embedding'] = df_keywords_list[f'keywords_yake_{column}_climatebert_embedding'].progress_apply(lambda x: np.array(x, dtype=np.float32))

100%|██████████| 77332/77332 [00:01<00:00, 42088.85it/s]


# Testing

In [ ]:
# Print out all keyword_yake_lemma for cpc_class_symbol = Y02W
print(df_keywords_titles[df_keywords_titles['cpc_class_symbol'].progress_apply(lambda x: 'Y02W' in x)][['keyword_yake_lemma', 'cpc_class_symbol']])

In [4]:
# Set search test to row with keyword = solar cells
df_search_test_titles = df_keywords_titles[df_keywords_titles[f'keyword_yake_lemma_{column}'] == 'renewable energy']

KeyError: 'keyword_yake_lemma_title'

In [ ]:
# Reset index of df_search_test_titles
df_search_test_titles.reset_index(drop=True, inplace=True)

In [ ]:
df_search_test_titles

# Similarity Search

In [5]:
import faiss
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors

# Euclidean Distance - kNN

In [ ]:
# Eucledian distance
index_bertforpatents = faiss.IndexFlatL2(1024)   # build the index
index_climatebert = faiss.IndexFlatL2(768)   # build the index
index_patentsberta = faiss.IndexFlatL2(768)   # build the index

# Add df_keywords_claims column 'keyword_yake_bertforpatents_embedding' to index
index_bertforpatents.add(np.array(df_keywords_list[f'keyword_yake_bertforpatents_embedding_{column}'].tolist()))
index_climatebert.add(np.array(df_keywords_list[f'keyword_yake_climatebert_embedding_{column}'].tolist()))
index_patentsberta.add(np.array(df_keywords_list[f'keyword_yake_patentsberta_embedding_{column}'].tolist()))

query_vector_bertforpatents = np.array(df_search_test_titles['keyword_yake_bertforpatents_embedding'][0]).reshape(1, -1)
query_vector_climatebert = np.array(df_search_test_titles['keyword_yake_climatebert_embedding'][0]).reshape(1, -1)
query_vector_patentsberta = np.array(df_search_test_titles['keyword_yake_patentsberta_embedding'][0]).reshape(1, -1)

In [ ]:
k = 25
D_bertforpatents, I_bertforpatents = index_bertforpatents.search(query_vector_bertforpatents, k)
D_climatebert, I_climatebert = index_climatebert.search(query_vector_climatebert, k)
D_patentsberta, I_patentsberta = index_patentsberta.search(query_vector_patentsberta, k)

In [ ]:
# Print row of df_keywords_claims that match the index
df_keywords_list.iloc[I_bertforpatents[0]]

# Cosine Similarity - kNN

In [ ]:
# Cosine similarity
index_bertforpatents = faiss.index_factory(1024, "Flat", faiss.METRIC_INNER_PRODUCT)
index_climatebert = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
index_patentsberta = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)

faiss.normalize_L2(np.array(df_keywords_list[f'keyword_yake_bertforpatents_embedding_{column}'].tolist()))
faiss.normalize_L2(np.array(df_keywords_list[f'keyword_yake_climatebert_embedding_{column}'].tolist()))
faiss.normalize_L2(np.array(df_keywords_list[f'keyword_yake_patentsberta_embedding_{column}'].tolist()))

index_bertforpatents.add(np.array(df_keywords_list[f'keyword_yake_bertforpatents_embedding_{column}'].tolist()))
index_climatebert.add(np.array(df_keywords_list[f'keyword_yake_climatebert_embedding_{column}'].tolist()))
index_patentsberta.add(np.array(df_keywords_list[f'keyword_yake_patentsberta_embedding_{column}'].tolist()))

query_vector_bertforpatents = np.array(df_search_test_titles[f'keyword_yake_bertforpatents_embedding_{column}'][0]).reshape(1, -1)
query_vector_climatebert = np.array(df_search_test_titles[f'keyword_yake_climatebert_embedding_{column}'][0]).reshape(1, -1)
query_vector_patentsberta = np.array(df_search_test_titles[f'keyword_yake_patentsberta_embedding_{column}'][0]).reshape(1, -1)

faiss.normalize_L2(query_vector_bertforpatents)
faiss.normalize_L2(query_vector_climatebert)
faiss.normalize_L2(query_vector_patentsberta)

In [ ]:
k = 6
D_bertforpatents, I_bertforpatents = index_bertforpatents.search(query_vector_bertforpatents, k)
D_climatebert, I_climatebert = index_climatebert.search(query_vector_climatebert, k)
D_patentsberta, I_patentsberta = index_patentsberta.search(query_vector_patentsberta, k)

In [ ]:
# Print row of df_keywords_claims that match the index
df_keywords_list.iloc[I_bertforpatents[0]]

# Cosine Similarity - Margin

In [26]:
# Convert lists of embeddings to a 2D array
bertforpatents_embeddings = np.vstack(df_keywords_list[f'keywords_yake_{column}_bertforpatents_embedding'].apply(np.array))
# climatebert_embeddings = np.vstack(df_keywords_list[f'keywords_yake_{column}_climatebert_embedding'].apply(np.array))
# patentsberta_embeddings = np.vstack(df_keywords_list[f'keywords_yake_{column}_patentsberta_embedding'].apply(np.array))

# Normalize embeddings
# bertforpatents_embeddings_normalized = normalize(bertforpatents_embeddings)
# climatebert_embeddings_normalized = normalize(climatebert_embeddings)
# patentsberta_embeddings_normalized = normalize(patentsberta_embeddings)

# Initialize NearestNeighbors with radius
radius = 0.25
nn_bertforpatents = NearestNeighbors(radius=radius, metric='cosine')
# nn_climatebert = NearestNeighbors(radius=radius, metric='cosine')
# nn_patentsberta = NearestNeighbors(radius=radius, metric='cosine')

# Fit the models with normalized embeddings
nn_bertforpatents.fit(bertforpatents_embeddings)
# nn_climatebert.fit(climatebert_embeddings)
# nn_patentsberta.fit(patentsberta_embeddings)

NearestNeighbors(metric='cosine', radius=0.25)

In [19]:
df_keywords_titles.head()

,keyword_yake_lemma,cleantech,keyword_yake_lemma_bertforpatents_embedding,neighbors_keyword_yake_lemma_bertforpatents_embedding,keywords_keyword_yake_lemma_bertforpatents_embedding
0,aberrant,1,"[0.2693934, 0.34846523, 0.6860294, 1.3137959, ...",[],[]
1,abundant,1,"[0.77166766, 0.18851693, 0.10953981, -0.604916...",[],[]
2,accumulator,1,"[0.50462705, -0.35983017, 0.38170147, 0.249620...",[406],[accumulator]
3,accurate,1,"[-0.7603557, 0.65543747, -0.50152314, -0.68986...",[423],[accurate]
4,acidic,1,"[-0.22680497, -0.63909197, 0.3487114, -0.08208...",[602],[acidic]


In [20]:
df_keywords_list.head()

,keywords_yake_claim_fulltext_lemma,abs_frequency,patent_id,publn_nr,oaid,keywords_yake_claim_fulltext_patentsberta_embedding,keywords_yake_claim_fulltext_climatebert_embedding,keywords_yake_claim_fulltext_bertforpatents_embedding
0,aaa,35,"[5626983, 9949165, 6492046, 5191275, 8758921, ...","[1142043, 3363790, 3041827, 1042828, 2982004]",[],"[-0.009164794300000001, -0.5787734985, -0.3139...","[-0.0193257574, -0.0141516225, 0.0131431855, 0...","[-0.4226518, -0.8338707, -0.44803128, 0.144316..."
1,aac,7,"[9067969, 7236059, 9524014, 6294352, 6417158, ...",[],[],"[-0.1441015601, -0.5350900292, -0.2306492031, ...","[0.025997683400000002, 0.0462871417, 0.0207896...","[0.48455638, -0.2590338, -0.6084336, 0.1693312..."
2,aar,5,"[5986108, 9458069]","[3103867, 2946009, 3385375]",[],"[-0.1567861736, -0.6682188511, -0.2236177772, ...","[-0.0596974939, 0.0217031986, 0.09825296700000...","[-0.3915314, 0.0007248279, -0.173878, 0.000474..."
3,aav,15,"[10119125, 9695220, 11104885, 10980896, 717289...","[3060575, 1135468, 3459965, 3062884]",[],"[-0.16056068240000002, -1.0209388733, -0.22905...","[-0.0716600269, -0.0178589281, 0.0750997365, -...","[0.0051265955, -0.9149596, -0.28275645, -0.387..."
4,aav capsid polypeptide,7,"[10119125, 9695220, 11104885, 10526584, 9719070]","[3060575, 3459965]",[],"[-0.1696463823, -0.8892566562, -0.203945100300...","[-0.0224281326, 0.1100657359, 0.04343590140000...","[-0.46955767, 0.11430413, -0.46052542, -0.5106..."


In [27]:
# Define the embeddings to process
# embeddings = [f'keyword_yake_bertforpatents_embedding', f'keyword_yake_climatebert_embedding', f'keyword_yake_patentsberta_embedding']
embeddings = ['keyword_yake_lemma_bertforpatents_embedding']
# nn_models = [nn_bertforpatents, nn_climatebert, nn_patentsberta]
nn_models = [nn_bertforpatents]
k_neighbors = 100  # Maximum number of neighbors to return

def get_keywords_by_indices(indices, df_source, k_neighbors=k_neighbors):
    return df_source.iloc[indices[:k_neighbors]][f'keywords_yake_{column}_lemma'].tolist() if len(indices) else []

def process_embeddings(row, nn_model, df_keywords_list, k_neighbors=k_neighbors):
    # query_vector = normalize(np.array([row]).reshape(1, -1))
    query_vector = np.array([row]).reshape(1, -1)
    distances, indices = nn_model.radius_neighbors(query_vector)
    neighbors = indices[0][:k_neighbors] if indices[0].size else np.array([])
    keywords = get_keywords_by_indices(neighbors, df_keywords_list, k_neighbors)
    return neighbors.tolist(), keywords

# # Process each embedding
# for embedding, nn_model in zip(embeddings, nn_models):
#     neighbors_col = f'neighbors_{embedding}'
#     keywords_col = f'keywords_{embedding}'

#     results = df_keywords_titles[embedding].progress_apply(lambda x: process_embeddings(x, nn_model, df_keywords_list, k_neighbors))
#     df_keywords_titles[neighbors_col] = results.progress_apply(lambda x: x[0])
#     df_keywords_titles[keywords_col] = results.progress_apply(lambda x: x[1])

def process_row(x):
    return process_embeddings(x, nn_model, df_keywords_list, k_neighbors)

pool = mp.Pool(mp.cpu_count())

for embedding, nn_model in zip(embeddings, nn_models):
    neighbors_col = f'neighbors_{embedding}'
    keywords_col = f'keywords_{embedding}'

    results = list(tqdm(pool.imap(process_row, df_keywords_titles[embedding]), total=df_keywords_titles[embedding].shape[0]))
    df_keywords_titles[neighbors_col] = [result[0] for result in results]
    df_keywords_titles[keywords_col] = [result[1] for result in results]

pool.close()

100%|██████████| 4777/4777 [02:55<00:00, 27.15it/s]


In [28]:
df_keywords_titles.to_json(f"/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/Similarity Search/df_keyword_{column}s_cosine_similarity_radius_{str(radius).replace('.','')}_neighbors_{str(k_neighbors)}_noun_chunks.json", orient='records')

In [42]:
len(df_keywords_titles)

2370

In [ ]:
df_keywords_titles.head()